In [1]:
import spacy 
import pyArango
import os
from os import path
import time
import glob
import pandas as pd
from arango import ArangoClient

Initialisation de la connection avec la base de données 

In [2]:
# Initialize the client for ArangoDB.
client = ArangoClient(hosts="http://localhost:8529")

In [3]:
sys_db = client.db('_system', username='root',password='root')

si besoin de repartir à zero:

In [96]:
sys_db.delete_database('text')

True

Check si la base de données existe déjà et création si besoin.

In [5]:
if not sys_db.has_database('text'):
    
    sys_db.create_database('text')
    db = client.db("text", username="root", password="root")
    graph = db.create_graph('text_explorer')
    # création des collections
    docs = graph.create_vertex_collection('docs')
    sentences = graph.create_vertex_collection('sentences')
    tokens = graph.create_vertex_collection('tokens')
    lemmas = graph.create_vertex_collection('lemmas')
    # création des arrêtes
    is_from = graph.create_edge_definition(
        edge_collection='is_from',
        from_vertex_collections=['sentences','tokens'],
        to_vertex_collections=['docs','sentences']
    )
    contracts_to = graph.create_edge_definition(
        edge_collection='contracts_to',
        from_vertex_collections=['tokens'],
        to_vertex_collections=['lemmas']
    )
    syntagmatic_link = graph.create_edge_definition(
        edge_collection='syntagmatic_link',
        from_vertex_collections=['tokens'],
        to_vertex_collections=['tokens']
    )
else:
    db = client.db("text", username="root", password="root")

L'objectif maintenant est de remplir ces champs avec les données extraites depuis le texte

Récupération des documents

In [6]:
def get_text(path):
    with open(path, encoding='utf8') as f:
        return(f.read().replace('\n',' '))
        f.close()

In [7]:
#dir_path = os.getcwd()
#dir_path

Liste des fichiers présents dans le dossier choisi pour l'analyse

In [8]:
files = glob.glob('/home/paul/projects/text_for_app/*.{}'.format('txt'))
files

['/home/paul/projects/text_for_app/jean_blog.txt',
 '/home/paul/projects/text_for_app/emploi étudiant et inégalités sociales.txt']

In [97]:
def get_filename_from_path(path):
    return os.path.normpath(path).split(os.sep)[-1]

In [98]:
documents = pd.DataFrame({'filepath':files,
                          'doc_name':[get_filename_from_path(filepath) for filepath in files],
                          'doc_number':list(range(0,len(files)))})
documents

,filepath,doc_name,doc_number
0,/home/paul/projects/text_for_app/jean_blog.txt,jean_blog.txt,0
1,/home/paul/projects/text_for_app/emploi étudia...,emploi étudiant et inégalités sociales.txt,1


In [99]:
def insert_docs(documents_found):
    in_db_doclist = pd.DataFrame(list(db.aql.execute('''FOR doc in docs RETURN doc''')))
    if in_db_doclist.shape == (0,0):
        pass
    else :
        documents_found = documents_found[~documents_found['doc_name'].isin(in_db_doclist['doc_name'])]
        last_doc_number = in_db_doclist['doc_number'].max()
        documents_found['doc_number'] = documents_found['doc_number'] + last_doc_number + 1
    
    dict_list_documents_to_insert = []
    for doc_name, number, path  in zip(documents_found['doc_name'], documents_found['doc_number'], documents_found['filepath']):
        dict_list_documents_to_insert.append({'_key':f'doc{number}',
                                                'doc_name':doc_name,
                                                'doc_path':path,
                                                'doc_number':number,
                                                'processed':'False'})
    db.collection('docs').import_bulk(dict_list_documents_to_insert)

In [100]:
insert_docs(documents)

# Chargement du modèle

In [13]:
nlp = spacy.load('fr_core_news_lg')

Traitement des fichiers par le modèle

## se renseigner sur comment faire du traitement en batch
    - mesurer l'empreinte mémoire de l'opération et ajuster combien de documents en même temps peuvent être traités

In [14]:
texts_to_process_df = pd.DataFrame(
                        list(
                            db.aql.execute('''
                                            FOR doc in docs
                                            FILTER doc.processed == 'False'
                                            RETURN {path :doc.doc_path, number : doc.doc_number}
                                            ''')
                            )
                        )
texts_to_process_df

,path,number
0,/home/paul/projects/text_for_app/jean_blog.txt,0
1,/home/paul/projects/text_for_app/emploi étudia...,1


Voir l'empreinte mémoire de l'opération. Découper l'output des textes à traiter pour ne pas tout fournir directement à l'opération ci dessous

In [15]:
processed_docs = list(nlp.pipe([get_text(path) for path in texts_to_process_df['path']]))

In [16]:
def insert_sentences(processed_doc, doc_number):
    dict_sentences_to_insert = []
    for sentence_number, sentence in enumerate(processed_doc.sents):
        if sentence.text != ' ':
            dict_sentences_to_insert.append({'_key':f'doc{doc_number}sent{sentence_number}',
                                             'content':sentence.text})
        else:
            pass
    db.collection('sentences').import_bulk(dict_sentences_to_insert)

In [17]:
for processed_text, doc_number in zip(processed_docs,texts_to_process_df['number']):
    insert_sentences(processed_text,doc_number)

Insertion des tokens et lemmas

Récupérer le vocabulaire token et lemma de chaque doc et les insérer dans la db

- clé = nombre arbitraire d'insertion
- valeur = le token ou lemma

Pour insérer de nouveaux mots par la suite faire une requette sur la table des tokens / lemma et insérer ceux qui ne sont pas déjà présents 

pour les relations faire une requete par phrase où l'on récupère chaque mot et sa clé associée
- relier ensuite les mots entre eux en utilisant les clés et la modalité de cette relation

Comment relier les phrases aux documents et les mots aux phrases ? 

### Extraction des correspondances lemmes / tokens depuis le texte

In [18]:
def get_vocab_table_from_text(processed_text):
    tokens, lemmas = [], []
    for token in processed_text:
        if not token.is_punct and not token.is_stop and not token.is_space and not token.is_digit:
            tokens.append(token.text.lower())
            lemmas.append(token.lemma_.lower())
    vocab_table = pd.DataFrame({'token':tokens,
                                'lemma':lemmas})
    return vocab_table

### Construction de deux tables contenant les occurences uniques des lemmes et tokens

In [19]:
def unique_vocabulary(token_lemma_table,word_type):
    return(token_lemma_table[word_type].drop_duplicates().reset_index(drop=True))

### Extraction du vocabulaire existant depuis la base de données

In [20]:
def get_vocab_in_db():
    vocab_tokens_from_db = pd.DataFrame(
        list(db.aql.execute("""
            FOR doc in tokens
            RETURN {word :doc.token, key : doc._key}
            """))
        )
        
    vocab_lemmas_from_db = pd.DataFrame(
        list(db.aql.execute("""
            FOR doc in lemmas
            RETURN {word :doc.lemma, key : doc._key}
            """))
        )
    return vocab_tokens_from_db, vocab_lemmas_from_db

### Comparaison du vocabulaire extrait du texte avec celui déjà contenu dans la db
- On ne rajoute que les instances jamais vues

In [21]:
def keep_only_new_vocab(vocab_from_text,vocab_from_db):
    new_vocab = vocab_from_text[~vocab_from_text.isin(vocab_from_db['word'])].reset_index(drop=True)
    return new_vocab

A refactoriser :

In [22]:
def add_vocab_to_db(processed_text):
    token_to_lemma_table = get_vocab_table_from_text(processed_text)
    
    text_tokens_series = unique_vocabulary(token_to_lemma_table,'token')
    text_lemmas_series = unique_vocabulary(token_to_lemma_table,'lemma')
    
    tokens_from_db, lemmas_from_db = get_vocab_in_db()
    
    if tokens_from_db.shape == (0,0):
        dict_list_tokens_to_insert = []
        for index, token in zip(text_tokens_series.index,text_tokens_series.values):
            dict_list_tokens_to_insert.append({"_key":f'token{index}',
                                               'token':token})
        db.collection('tokens').import_bulk(dict_list_tokens_to_insert)

        dict_list_lemmas_to_insert = []
        for index, lemma in zip(text_lemmas_series.index,text_lemmas_series.values):
            dict_list_lemmas_to_insert.append({"_key":f'lemma{index}',
                                               'lemma':lemma})
        db.collection('lemmas').import_bulk(dict_list_lemmas_to_insert)

    else :
        new_vocab_tokens = keep_only_new_vocab(text_tokens_series,tokens_from_db)
        new_vocab_lemmas = keep_only_new_vocab(text_lemmas_series,lemmas_from_db)

        new_vocab_tokens.index = new_vocab_tokens.index + tokens_from_db.shape[0] + 1
        new_vocab_lemmas.index = new_vocab_lemmas.index + lemmas_from_db.shape[0] + 1

        dict_list_tokens_to_insert = []
        for index, token in zip(new_vocab_tokens.index,new_vocab_tokens.values):
            dict_list_tokens_to_insert.append({"_key":f'token{index}',
                                               'token':token})
        db.collection('tokens').import_bulk(dict_list_tokens_to_insert)

        dict_list_lemmas_to_insert = []
        for index, lemma in zip(new_vocab_lemmas.index,new_vocab_lemmas.values):
            dict_list_lemmas_to_insert.append({"_key":f'lemma{index}',
                                               'lemma':lemma})
        db.collection('lemmas').import_bulk(dict_list_lemmas_to_insert)

In [23]:
get_vocab_table_from_text(processed_docs[0])

,token,lemma
0,faire,faire
1,inutilement,inutilement
2,durer,durer
3,suspense,suspense
4,réponse,réponse
...,...,...
106,évoluer,évoluer
107,cesse,cesse
108,jamais,jamais
109,étonner,étonner


In [24]:
add_vocab_to_db(processed_docs[0])

In [25]:
add_vocab_to_db(processed_docs[1])

### Relations

Les 3 types de relations dans le graphe, ce qu'elles connectent et ce qu'elles contiennent

phrases aux docs :
- faire un call sur les phrases de la db 
- clé from = la clé de chaque phrase 
- clé to = la première partie de la clé 

In [26]:
sentences_keys = pd.Series(list(db.aql.execute('''
                        FOR doc in sentences
                        return doc._key
                        ''')))

In [27]:
doc_number_for_sentences = sentences_keys.str.extract('(\d+)')[0]
sentences_number = sentences_keys.str.extract('\D+\d+\D+(\d+)')[0]

In [28]:
sentences_keys

0       doc0sent0
1       doc0sent1
2      doc0sent11
3      doc0sent12
4      doc0sent13
          ...    
531    doc1sent95
532    doc1sent96
533    doc1sent97
534    doc1sent98
535    doc1sent99
Length: 536, dtype: object

is_from :
- les phrases aux documents

In [29]:
dict_is_from_sent_doc_to_insert = []
for sentence_key, doc_number, sentence_number in zip(sentences_keys.values,doc_number_for_sentences,sentences_number.values):
    dict_is_from_sent_doc_to_insert.append({'_from':sentence_key,
                                            '_to':f'doc{doc_number}',
                                            'sentence_number': sentence_number})
db.collection('is_from').import_bulk(dict_is_from_sent_doc_to_insert,
                                     from_prefix='sentences/',
                                     to_prefix='docs/')

{'error': False,
 'created': 536,
 'errors': 0,
 'empty': 0,
 'updated': 0,
 'ignored': 0,
 'details': []}

is_from :
- les tokens aux phrases

Extraction d'une forme tabulaire de la structure syntagmatique.
  - Le processus est itéré sur l'ensemble des phrases du document

In [30]:
def create_dependancy_df(processed_text,doc_number):
    token_text, token_dep, token_head_text, token_head_pos, sentence_number = [], [], [], [],[]

    for count, sentence in enumerate(processed_text.sents):
        for token in sentence:
            if not token.is_punct and not token.is_stop and not token.is_space and not token.is_digit:
                token_text.append(token.text)
                token_dep.append(token.dep_), 
                token_head_text.append(token.head.text), 
                token_head_pos.append( token.head.pos_)
                sentence_number.append(f'doc{doc_number}sent{count}')

    
    df = pd.DataFrame({'token':token_text,
                       'dep':token_dep,
                       'head_text':token_head_text,
                       'head_pos':token_head_pos,
                       'sentence_number':sentence_number})   
    df = df[df['head_pos']!="SPACE"]
    
    return df

In [31]:
def create_dependancy_df(processed_text,doc_number):
    token_text, token_lemma, token_dep, token_head_text, token_head_pos, sentence_number = [], [], [], [], [], []

    for count, sentence in enumerate(processed_text.sents):
        for token in sentence:
            if not token.is_punct and not token.is_stop and not token.is_space and not token.is_digit:
                token_text.append(token.text)
                token_lemma.append(token.lemma_)
                token_dep.append(token.dep_), 
                token_head_text.append(token.head.text), 
                token_head_pos.append( token.head.pos_)
                sentence_number.append(f'doc{doc_number}sent{count}')

    
    df = pd.DataFrame({'token':token_text,
                       'lemma':token_lemma,
                       'dep':token_dep,
                       'head_text':token_head_text,
                       'head_pos':token_head_pos,
                       'sentence_number':sentence_number})   
    df = df[df['head_pos']!="SPACE"]
    
    return df

In [65]:
dependancy_df = create_dependancy_df(processed_docs[0],0) 
dependancy_df

,token,lemma,dep,head_text,head_pos,sentence_number
0,faire,faire,xcomp,vais,VERB,doc0sent0
1,inutilement,inutilement,advmod,faire,VERB,doc0sent0
2,durer,durer,xcomp,faire,VERB,doc0sent0
3,suspense,suspense,obj,durer,VERB,doc0sent0
4,réponse,réponse,nsubj,oui,ADV,doc0sent0
...,...,...,...,...,...,...
106,évoluer,évoluer,xcomp,peuvent,VERB,doc0sent15
107,cesse,cesse,ROOT,cesse,VERB,doc0sent16
108,jamais,jamais,advmod,cesse,VERB,doc0sent16
109,étonner,étonner,xcomp,cesse,VERB,doc0sent16


In [34]:
vocab_table_tokens_in_db = get_vocab_in_db()[0]
vocab_table_tokens_in_db

,word,key
0,faire,token0
1,inutilement,token1
2,durer,token2
3,suspense,token3
4,réponse,token4
...,...,...
2855,florence,token2856
2856,lefresne,token2857
2857,vecteurs,token2858
2858,structurelle,token2859


In [79]:
token_from_sentence_table = dependancy_df.merge(vocab_table_tokens_in_db, 
                                                        left_on='token', 
                                                        right_on='word', sort = 'outer')\
                                                        .rename(columns={'key':'token_key'})\
                                                        .drop('word',axis=1)

In [80]:
token_from_sentence_table

,token,lemma,dep,head_text,head_pos,sentence_number,token_key
0,aberration,aberration,conj,est,VERB,doc0sent7,token33
1,aborder,aborder,acl,manière,NOUN,doc0sent15,token85
2,aime,aime,ROOT,aime,VERB,doc0sent7,token35
3,air,air,obj,donne,VERB,doc0sent1,token13
4,ans,an,obl:mod,commence,VERB,doc0sent12,token61
...,...,...,...,...,...,...,...
106,vêtement,vêtement,ROOT,vêtement,NOUN,doc0sent12,token45
107,vêtement,vêtement,nsubj,peut,VERB,doc0sent12,token45
108,vêtement,vêtement,obj,aborder,VERB,doc0sent15,token45
109,étonner,étonner,xcomp,cesse,VERB,doc0sent16,token90


In [78]:
dependancy_table_for_insert

,token,lemma,dep,head_text,head_pos,sentence_number,token_key,head_text_key
0,perverse,pervers,amod,aberration,NOUN,doc0sent7,token34,token33
1,vêtement,vêtement,obj,aborder,VERB,doc0sent15,token45,token85
2,aime,aime,ROOT,aime,VERB,doc0sent7,token35,token35
3,beaucoup,beaucoup,advmod,aime,VERB,doc0sent7,token36,token35
4,porter,porter,xcomp,aime,VERB,doc0sent7,token37,token35
...,...,...,...,...,...,...,...,...
77,valeur,valeur,obl:mod,trouve,VERB,doc0sent13,token49,token67
78,pareil,pareil,amod,truc,NOUN,doc0sent3,token24,token23
79,plan,plan,nmod,valeur,NOUN,doc0sent13,token70,token49
80,symbolique,symbolique,conj,vêtement,NOUN,doc0sent15,token86,token45


In [53]:
token_from_sentence_table = dependancy_df.merge(vocab_table_tokens_in_db, 
                                                        left_on='token', 
                                                        right_on='word')\
                                                        .rename(columns={'key':'token_key'})\
                                                        .drop('word',axis=1)

dependancy_table_for_insert = token_from_sentence_table.merge(vocab_table_tokens_in_db,
                                                        left_on='head_text',
                                                        right_on='word')\
                                                        .rename(columns={'key':'head_text_key'})\
                                                        .drop('word',axis=1)

In [36]:
dict_is_from_sent_token_to_insert = []
for token_key,sentence_number in zip(token_from_sentence_table['token_key'],token_from_sentence_table['sentence_number'] ):
    dict_is_from_sent_token_to_insert.append({'_from':token_key,
                                            '_to':sentence_number})
db.collection('is_from').import_bulk(dict_is_from_sent_token_to_insert,
                                     from_prefix='tokens/',
                                     to_prefix='sentences/')

syntagmatic_link :
- les tokens aux tokens 
    - le type de relation grammaticale
    - l'ID de la phrase contenant cette relation

Merge des tokens avec les id correspondants dans la db

In [ ]:
get_vocab_in_db()[0]

,word,key
0,faire,token0
1,inutilement,token1
2,durer,token2
3,suspense,token3
4,réponse,token4
...,...,...
2855,florence,token2856
2856,lefresne,token2857
2857,vecteurs,token2858
2858,structurelle,token2859


In [42]:
dependancy_table_for_insert

,token,lemma,dep,head_text,head_pos,sentence_number,token_key,head_text_key
0,faire,faire,xcomp,vais,VERB,doc0sent0,token0,token0
1,faire,faire,ROOT,faire,VERB,doc0sent9,token0,token0
2,inutilement,inutilement,advmod,faire,VERB,doc0sent0,token1,token1
3,durer,durer,xcomp,faire,VERB,doc0sent0,token2,token2
4,suspense,suspense,obj,durer,VERB,doc0sent0,token3,token3
...,...,...,...,...,...,...,...,...
106,évoluer,évoluer,xcomp,peuvent,VERB,doc0sent15,token87,token87
107,cesse,cesse,ROOT,cesse,VERB,doc0sent16,token88,token88
108,jamais,jamais,advmod,cesse,VERB,doc0sent16,token89,token89
109,étonner,étonner,xcomp,cesse,VERB,doc0sent16,token90,token90


In [59]:
dict_syntagmatic_link_to_insert = []
for head_text_key, token_key, dep_relation, head_pos_tag, sentence_number in zip(dependancy_table_for_insert['head_text_key'],
                                                                                 dependancy_table_for_insert['token_key'],
                                                                                 dependancy_table_for_insert['dep'],
                                                                                 dependancy_table_for_insert['head_pos'],
                                                                                 dependancy_table_for_insert['sentence_number']):
    dict_syntagmatic_link_to_insert.append({'_from':head_text_key,
                                            '_to':token_key,
                                            'dep_relation':dep_relation,
                                            'head_pos_tag':head_pos_tag,
                                            'from_sentence_number':sentence_number})
db.collection('syntagmatic_link').import_bulk(dict_syntagmatic_link_to_insert,
                                             from_prefix='tokens/',
                                             to_prefix='tokens/')

{'error': False,
 'created': 82,
 'errors': 0,
 'empty': 0,
 'updated': 0,
 'ignored': 0,
 'details': []}

contracts_to :
- Les tokens aux lemmes

In [40]:
lemmas_in_db = get_vocab_in_db()[1]

In [91]:
contracts_to_table = dependancy_table_for_insert.merge(lemmas_in_db, left_on='lemma', right_on='word')\
                        .rename(columns={'key':'lemma_key'})\
                        .drop('word',axis=1)

In [94]:
contracts_to_table

,token,lemma,dep,head_text,head_pos,sentence_number,token_key,head_text_key,lemma_key
0,perverse,pervers,amod,aberration,NOUN,doc0sent7,token34,token33,lemma34
1,vêtement,vêtement,obj,aborder,VERB,doc0sent15,token45,token85,lemma45
2,vêtement,vêtement,obj,porte,VERB,doc0sent11,token45,token44,lemma45
3,vêtement,vêtement,nmod,style,NOUN,doc0sent11,token45,token46,lemma45
4,vêtement,vêtement,ROOT,vêtement,NOUN,doc0sent12,token45,token45,lemma45
...,...,...,...,...,...,...,...,...,...
77,principal,principal,amod,sujet,NOUN,doc0sent12,token54,token53,lemma54
78,trouve,trouve,ROOT,trouve,VERB,doc0sent13,token67,token67,lemma67
79,pareil,pareil,amod,truc,NOUN,doc0sent3,token24,token23,lemma24
80,plan,plan,nmod,valeur,NOUN,doc0sent13,token70,token49,lemma70


In [95]:
dict_contracts_to = []
for token_key, lemma_key, sentence_number in zip(contracts_to_table['token_key'],
                                                 contracts_to_table['lemma_key'],
                                                 contracts_to_table['sentence_number']):
    
    dict_contracts_to.append({'_from':token_key,
                             '_to':lemma_key,
                             'sentence_number':sentence_number})
    
db.collection('contracts_to').import_bulk(dict_contracts_to,
                                             from_prefix='tokens/',
                                             to_prefix='lemmas/')

{'error': False,
 'created': 82,
 'errors': 0,
 'empty': 0,
 'updated': 0,
 'ignored': 0,
 'details': []}

# TODO :
- Insérer une logique de check database / non répétition des insertions lors de l'insertion de nouvelles phrases
- Assigner dans is_from entre les mots et les phrases la clé de la phrase (doc0sent20) -check
- connecter les lemmes avec les tokens

In [57]:
pd.DataFrame(dict_syntagmatic_link_to_insert)

,_from,_to,dep_relation,head_pos_tag,from_sentence_number
0,token0,token0,ROOT,VERB,doc0sent9
1,token0,token1,advmod,VERB,doc0sent0
2,token0,token2,xcomp,VERB,doc0sent0
3,token0,token38,advmod,VERB,doc0sent9
4,token0,token39,obj,VERB,doc0sent9
...,...,...,...,...,...
77,token84,token85,acl,NOUN,doc0sent15
78,token88,token88,ROOT,VERB,doc0sent16
79,token88,token89,advmod,VERB,doc0sent16
80,token88,token90,xcomp,VERB,doc0sent16
